In [1]:
import cv2
from cv2 import aruco
import numpy as np
import math
import time
import os
import sys
#from IPython.display import clear_output
#import multiprocessing
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
import traceback
#import matplotlib as mpl
import pandas as pd
#%matplotlib nbagg

In [2]:
def make_complementary_color_bgr(somecolor=type(tuple)):

    complementary_color = [0,0,0]
    for i in range(len(somecolor)):
        complementary_color[i] = 255 - somecolor[i]
    
    complementary_color = tuple(complementary_color)

    return complementary_color

In [3]:
def create_tag_list(df):
    try:
        tag_list = list(df['ID'].unique())
        print("tag_list object created")
        print(tag_list)
        return tag_list
    except:
        print("Check out your csv file, does it have bees in it? Ie does it have a \'ID\' column?")
        return None

In [4]:
def create_tag_colors(tag_colors, tag_list):
    if isinstance(tag_colors, type(None)): 
            if not isinstance(tag_list, type(None)) and len(tag_list) > 0:
                cm = plt.get_cmap('rainbow') # or 'gist_rainbow'
                cNorm  = colors.Normalize(vmin=0, vmax=len(tag_list)-1)
                scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
                color_palette=[scalarMap.to_rgba(i, bytes=True) for i in range(len(tag_list))]
                tag_colors = pd.Series(color_palette, index=tag_list)
                return tag_colors
            else:
                print("Had an issue setting the colors for the tags...")
                return None
            
    else:
         if len(tag_colors) == len(tag_list):
            return tag_colors
         
         else:
            print("Had an issue setting the colors for the tags...")
            return None
            

In [5]:
def create_bee_pairs_list(tag_list):
    if len(tag_list) > 0:
        bee_pairs = [(a, b) for idx, a in enumerate(tag_list) for b in tag_list[idx + 1:]] # splits list into all its pair combinations. Ex list [3,4,5] becomes [(3,4),(3,5),(4,5)]
        return bee_pairs

In [6]:
def create_line_colors(line_colors, bee_pairs):

    if len(bee_pairs) > 0:

        if isinstance(line_colors, type(None)): 

            cm1 = plt.get_cmap('gist_earth') # or 'gist_rainbow'
            cNorm1  = colors.Normalize(vmin=0, vmax=len(bee_pairs)-1)
            scalarMap1 = mplcm.ScalarMappable(norm=cNorm1, cmap=cm1)
            color_palette1=[scalarMap1.to_rgba(i, bytes=True) for i in range(len(bee_pairs))]
            line_colors = pd.Series(color_palette1, index=bee_pairs)
            return line_colors
        
        elif len(line_colors) == len(bee_pairs):
            return line_colors

        else:
            print("Had an issue setting the colors for the distance lines between bees...")
            return None
    else:
        print("No bee pairs to create line colors for...")
        return None

In [7]:
def extract_filename(video_path):
    file_dir, file = os.path.split(video_path)
    filename, extension = os.path.splitext(file)
    print(filename)
    return filename, extension


In [8]:
def extract_size(vid):
    print("write_video is on")
    frame_width = int(vid.get(3))
    frame_height = int(vid.get(4))
    size = (frame_width, frame_height)
    return size


In [9]:
def create_outputfile_and_videowriter(tracked_video_destination_directory, filename, video_filename_addition, extension, playback_framerate, size):

    if video_filename_addition == None:
            video_filename_addition = ''
            
    if extension == '.mjpeg':
        output_file = "{0}{1}{2}{3}".format(tracked_video_destination_directory, filename, video_filename_addition, '.mjpeg')
        print(output_file)
        out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc('m','j','p','g'), playback_framerate, size) # .format(tracked_video_destination_directory, filename, video_filename_addition)
        return out
    
    elif extension == '.mp4':
        output_file = "{0}{1}{2}{3}".format(tracked_video_destination_directory, filename, video_filename_addition, '.mp4')
        print(output_file)
        out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc('m','p','4','v'), playback_framerate, size)
        return out
    
    else:
        print("This script can only process mjpeg and mp4 files")
        return None



In [10]:
def find_bee_pair_coordinates(i, bee_pairs, frame_ids, frame_df):

    bee1=bee_pairs[i][0]
    bee2=bee_pairs[i][1]
    
    if bee1 in frame_ids and bee2 in frame_ids:
        bee1row = frame_df[ frame_df['ID']== bee1 ]
        xmeanbee1 = bee1row['centroidX']
        xmeanbee1 = int(xmeanbee1.iloc[0])
        ymeanbee1 = bee1row['centroidY']
        ymeanbee1 = int(ymeanbee1.iloc[0])

        bee1_loc = (xmeanbee1, ymeanbee1)
        
        bee2row = frame_df[ frame_df['ID']== bee2 ]
        xmeanbee2 = bee2row['centroidX']
        xmeanbee2 = int(xmeanbee2.iloc[0])
        ymeanbee2 = bee2row['centroidY']
        ymeanbee2 = int(ymeanbee2.iloc[0])

        bee2_loc = (xmeanbee2, ymeanbee2)

        distance = math.sqrt( ((xmeanbee1-xmeanbee2)**2)+((ymeanbee1-ymeanbee2)**2) )

        return bee1, bee2, bee1_loc, bee2_loc, distance
    
    else:
        return bee1, bee2, None, None, None

    

In [20]:
def draw_bee_lines(i, pairwise_focal_bee, focal_bee_coord, bee1, bee2, bee1_loc, bee2_loc, pairwise_line_size, focal_bee_color, background_bee_color, line_colors, bee_pairs, frame, frame_num, pairwise_trail, pairwise_trail_list, focal_bee_container, distance, contacts, contact_distance, contacts_container, queen_id, queen_contact_distance):
    
    #Limit tracking bee lines to a certain distance between bees
    if distance > 800:
        return frame, focal_bee_coord, focal_bee_container, pairwise_trail_list, contacts_container
    #first check if bees are in contact with each other. If yes, store the result and exit the function - these lines will be drawn later (on top of other lines)
    if contacts == 'ON':
        
        if int(bee1) == int(queen_id) or int(bee2) == int(queen_id):
            #print(f"Making contact distance ({contact_distance}) queen contact distance: {queen_contact_distance}")
            contact_distance = queen_contact_distance
            #print(f"New contact distance: {contact_distance}")
            
        else:
            contact_distance = contact_distance

        if distance <= contact_distance:
            #if bee1 == 17 or bee2 == 17:
            print("Contact between bees {0} and {1} at frame {2}: distance is {3}, contact distance is {4}".format(bee1, bee2, frame_num, distance, contact_distance))
            contacts_container.append([bee1, bee2, bee1_loc, bee2_loc])
            return frame, focal_bee_coord, focal_bee_container, pairwise_trail_list, contacts_container


    #Next, if we are tracking a focal bee for pairwise distance tracking, check if its either of the current pairs
    if type(pairwise_focal_bee) == int:
        if bee1 == pairwise_focal_bee and focal_bee_coord == None:
            focal_bee_coord = bee1_loc 

        elif bee2 == pairwise_focal_bee and focal_bee_coord == None:
            focal_bee_coord = bee2_loc

        #if either is the focal bee, store the data to be drawn later (on top of other lines, but before the contact lines)
        if bee1 == pairwise_focal_bee or bee2 == pairwise_focal_bee:
            focal_bee_container.append([bee1_loc, bee2_loc, focal_bee_color, pairwise_line_size]) #add to list so that they're tracked on last, ie will be on top

        #if neither is the focal bee, draw the lines with the background bee color    
        else:
            cv2.line(frame, bee1_loc, bee2_loc, background_bee_color, pairwise_line_size)

    #if we are not tracking a focal bee, draw the normal line colors for that bee pair
    else:
        line_color = line_colors[bee_pairs[i]]
        line_color_rgb = ( int(line_color[0]), int(line_color[1]), int(line_color[2]) )

        if pairwise_trail == 'ON':
            pairwise_trail_list.append([frame_num, bee1_loc, bee2_loc, line_color_rgb, pairwise_line_size]) # append current line to list

        else:
            cv2.line(frame, bee1_loc, bee2_loc, line_color_rgb, pairwise_line_size)

    return frame, focal_bee_coord, focal_bee_container, pairwise_trail_list, contacts_container
    

In [12]:
def plot_focal_bee_lines(frame, focal_bee_container, focal_bee_coord, focal_bee_color, avg_distance_vector, avg_distance_vector_color):

    avg_distance_vector_data = []
    for row in focal_bee_container:
        cv2.line(frame, row[0], row[1], row[2], row[3]) # row = [bee1_loc, bee2_loc, line_color_rgb, line size]

        # if plotting the avg distance vector, finds the bees coordinates that arent the focal bee in the focal_bee_container
        if avg_distance_vector == 'ON':
            if row[0] == focal_bee_coord:
                avg_distance_vector_data.append([row[1]])
                
            elif row[1] == focal_bee_coord:
                avg_distance_vector_data.append([row[0]])

    # if plotting the avg dinstance vector, finds the average coordinates of all other bees 
    if avg_distance_vector == 'ON':
        print(avg_distance_vector_data)
        avg_dvd_nparray = np.array(avg_distance_vector_data)

        # below code removes the 2nd dimension that is equal to 1 (this is an uneccesary dimension for us), in this case will make it array go from (x, 1, z) to (x,y)

        if avg_dvd_nparray.shape[0] != 1: # if more than just one set of coordinates in avg_distance_vector_data...
            #print('using np.squeeze...')
            #print(avg_dvd_nparray)
            #print(avg_dvd_nparray.shape)
            avg_dvd_nparray = avg_dvd_nparray.squeeze() # removes and dimension equal to 1 (this is an uneccesary dimension for us), in this case will make it array go from (x, 1, z) to (x,y)
            #print(avg_dvd_nparray.shape)
        else:                             # if only one set of coordinates in avg_distance_vector_data
            print('shouldnt use np.squeeze...')
            print(avg_dvd_nparray)
            print(avg_dvd_nparray.shape)
            x_column = avg_dvd_nparray[0][0][0]
            y_column = avg_dvd_nparray[0][0][1]
            avg_dvd_nparray = np.column_stack((x_column, y_column))
            print(avg_dvd_nparray)
            print(avg_dvd_nparray.shape)
    
        # removes and dimension equal to 1 (this is an uneccesary dimension for us), in this case will make it array go from (x, 1, z) to (x,y)
        # note - got this error for the below line: Exception: too many indices for array: array is 1-dimensional, but 2 were indexed, Type: <class 'IndexError'>, line number: 144
        avg_dv_x = np.mean(avg_dvd_nparray[:,0])
        avg_dv_y = np.mean(avg_dvd_nparray[:,1])
        avg_dv_xy = (int(avg_dv_x), int(avg_dv_y))
        if avg_distance_vector_color == 'complementary':
            complementary = make_complementary_color_bgr(focal_bee_color) # use the complement color in bgr space to plot the average distance vector per frame
            cv2.arrowedLine(frame, focal_bee_coord, avg_dv_xy, complementary, 15, tipLength=0.05)
        else:
            cv2.arrowedLine(frame, focal_bee_coord, avg_dv_xy, avg_distance_vector_color, 15, tipLength=0.05)

    return frame

In [13]:
def draw_pairwise_trail_lines(frame, pairwise_trail_list, frame_num):
    for row in pairwise_trail_list:
        if frame_num - row[0] < 10: # draw lines over the past 10 frames
            # row = [bee1_loc, bee2_loc, line_color_rgb, line size]
            cv2.line( frame, row[1], row[2], row[3], row[4] )
    
    return frame

In [14]:
def draw_contact_lines(contacts_container, contacts_color, contacts_line_size, enable_contacts_alpha_channel, contacts_alpha, frame, updated_frame_copy):
    
    for contact in contacts_container: # contact row = [bee1, bee2, bee1_loc, bee2_loc]
        if contact[0] == 17 or contact[1] == 17:
            print("contact for bee 17")
        cv2.line(frame, contact[2], contact[3], contacts_color, contacts_line_size)

    #Addweighted uses the alpha value to blend the image with the contacts drawn on top of it to an image saved just prior to the contacts being drawn
    if enable_contacts_alpha_channel == 'ON':
        frame = cv2.addWeighted(frame, contacts_alpha, updated_frame_copy, 1 - contacts_alpha, 0)
    
    return frame

In [15]:
def draw_ids(frame, id, frame_df, tag_list, tag_colors, contacts, contacts_container, contacts_color):

    if tag_list != None and id in tag_list: #at this point tag_list should have a value, even if originally set to None
        #find id x and y coordinates
        id_row = frame_df[ frame_df['ID'] == id ]
        xmean = id_row['centroidX'] #int(ser.iloc[0])
        xmean = int(xmean.iloc[0])
        ymean = id_row['centroidY']
        ymean = int(ymean.iloc[0])

        bee_color = tag_colors[id]
        bee_color_rgb = ( int(bee_color[0]), int(bee_color[1]), int(bee_color[2]) )
    
        #if contacts are on, Determine whether bee is in contact with another bee
        in_contact = False
        if contacts == 'ON':
            for contact in contacts_container:
                if id == contact[0] or id == contact[1]:
                    in_contact = True
                    break
            
            if in_contact == True:
                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (xmean - 40, ymean - 20)
                fontScale = 2
                thickness = 3
                frame = cv2.putText(frame, str(id), org, font, fontScale, (0,0,0), thickness*2, cv2.LINE_AA) #create a black border around text (w doubled thickness)
                frame = cv2.putText(frame, str(id), org, font, fontScale, contacts_color, thickness, cv2.LINE_AA)
            
            else:
                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (xmean - 40, ymean - 20)
                fontScale = 2
                color = bee_color_rgb
                thickness = 3
                frame = cv2.putText(frame, str(id), org, font, fontScale, (0,0,0), thickness*2, cv2.LINE_AA) #create a black border around text (w doubled thickness)
                frame = cv2.putText(frame, str(id), org, font, fontScale, color, thickness, cv2.LINE_AA)

        else:
            font = cv2.FONT_HERSHEY_SIMPLEX
            org = (xmean - 40, ymean - 20)
            fontScale = 2
            color = bee_color_rgb
            thickness = 3
            frame = cv2.putText(frame, str(id), org, font, fontScale, (0,0,0), thickness*2, cv2.LINE_AA) #create a black border around text (w doubled thickness)
            frame = cv2.putText(frame, str(id), org, font, fontScale, color, thickness, cv2.LINE_AA)

        return frame
    
    else:
        print("id not in tag_list or tag list None")
        return frame

        

In [16]:
def draw_centroids_and_tracking_trails(frame, id, tag_list, frame_df, tracking_trail, tracking_trail_dict, tag_colors, frame_num, contacts, contacts_color, non_contact_color, contacts_container ):
    
    frame_copy = frame.copy()
    
    if tag_list != None and id in tag_list: #at this point tag_list should have a value, even if originally set to None
        #find id x and y coordinates
        id_row = frame_df[ frame_df['ID'] == id ]
        xmean = id_row['centroidX'] #int(ser.iloc[0])
        xmean = int(xmean.iloc[0])
        ymean = id_row['centroidY']
        ymean = int(ymean.iloc[0])
    
        frame_index = str(id) + '_index'
        if id not in tracking_trail_dict.keys():
            tracking_trail_dict[id] = [(xmean, ymean)]
            tracking_trail_dict[frame_index] = [frame_num]
        else:
            tracking_trail_dict[id] += [(xmean, ymean)]
            tracking_trail_dict[frame_index] += [frame_num]

        #if contacts are on, Determine whether bee is in contact with another bee
        in_contact = False
        if contacts == 'ON':
            for contact in contacts_container:
                if id == contact[0] or id == contact[1]:
                    in_contact = True
                    break

#end visualizing tag ids
# start visualizing tag dots (and trails if applicable)                           

        bee_color = tag_colors[id]
        bee_color_rgb = ( int(bee_color[0]), int(bee_color[1]), int(bee_color[2]) )

        if contacts == 'ON':
            bee_color_rgb = non_contact_color

#start of code to draw lines between tags detected in the last 10 frames

        if tracking_trail == 'ON':
            trail = tracking_trail_dict[id] # save data associated with tag id in the dictionary
            trail_index = tracking_trail_dict[frame_index]
            for i in range(len(trail)):
                if (frame_num - trail_index[i]) < 11: # if current frame - frame i is less than 11
                    trail = trail[i:] # reassign trail the subset of trail data from row i onwards
                    trail_index = trail_index[i:] # reassign trail_index the subset of trail_index data from row i onwards
                    break
            
            #We want the tracking trail to diminish in intensity the further away from the current frame it is
            #So we're assigning an alpha value to each frame in the trail based on its distance from the current frame
            alpha_values = {9: 0.1, 8: 0.2, 7: 0.3, 6: 0.4, 5: 0.5, 4: 0.6, 3: 0.7, 2: 0.8, 1: 0.9, 0: 1.0}
            
            for i in range(len(trail)):
                if i == 0:
                    continue
                print(i)

                #if tracking trail and contacts are on and the current tag id is in the contacts container, draw the dot in the contacts color
                if contacts == 'ON' and in_contact == True:
                    cv2.line(frame, trail[i-1], trail[i], bee_color_rgb, 8)
                    cv2.circle(frame, tuple([int(xmean), int(ymean)]), 20, contacts_color, -1)
                else:
                    #Based on the distance of the frame from the current frame, assign it the appropriate alpha value
                    print(f"trail_index[i]: {trail_index[i]}")
                    print(f"frame_num: {frame_num}")
                    frame_distance = frame_num - trail_index[i]
                    print(f"frame distance: {frame_distance}")
                    print(f"alpha_value: {alpha_values[frame_distance]}")
                    alpha = alpha_values[frame_distance]

                    cv2.line(frame, trail[i-1], trail[i], bee_color_rgb, 8)
                    cv2.circle(frame, tuple([int(xmean), int(ymean)]), 20, bee_color_rgb, -1)
                    frame = cv2.addWeighted(frame, alpha, frame_copy, 1 - alpha, 0)

        elif contacts == 'ON' and in_contact == True:
            cv2.circle(frame, tuple([int(xmean), int(ymean)]), 20, contacts_color, -1)
                
        else:
            cv2.circle(frame, tuple([int(xmean), int(ymean)]), 20, bee_color_rgb, -1)

        return frame, tracking_trail_dict

In [17]:
def print_error_message(e):
    traceback.print_exc()
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print("Exception: {}, Type: {}, line number: {}".format(e, exc_type, exc_tb.tb_lineno)) #exc_type, exc_obj, exc_tb.tb_lineno
    print("issue detecting markers")

In [21]:
# switched variable name to show_video - need to change this in other scripts!
def write_tags_onto_video(csv_path, video_path, tracked_video_destination_directory, playback_framerate=10,
                                                                                     show_video='ON', 
                                                                                     write_video='ON',  
                                                                                     tracking_trail='OFF',
                                                                                     pairwise_distance_tracking='OFF',
                                                                                     pairwise_trail='OFF',
                                                                                     pairwise_focal_bee=None,
                                                                                     pairwise_line_size=4,
                                                                                     enable_pairwise_alpha_channel='OFF',
                                                                                     pairwise_alpha=0.6,
                                                                                     focal_bee_color=None,
                                                                                     background_bee_color=None,
                                                                                     contacts = 'OFF',
                                                                                     contact_distance=200,
                                                                                     queen_id=1,
                                                                                     queen_contact_distance=None,
                                                                                     contacts_color=None,
                                                                                     non_contact_color=None,
                                                                                     contacts_line_size=6,
                                                                                     enable_contacts_alpha_channel='OFF',
                                                                                     contacts_alpha=0.8,
                                                                                     avg_distance_vector='OFF',
                                                                                     avg_distance_vector_color='complementary',
                                                                                     tag_list = None,
                                                                                     tag_colors = None,
                                                                                     line_colors = None, 
                                                                                     video_filename_addition=None ):
    #read in csv data and video
    df = pd.read_csv(csv_path)
    vid = cv2.VideoCapture(video_path)

    #create tag list from csv data if none is provided
    if tag_list == None:
        tag_list = create_tag_list(df)
    #create tag colors if none are provided or if the length of the tag list is different from the length of the tag colors list
    if show_video == 'ON' or write_video == 'ON':
        tag_colors = create_tag_colors(tag_colors, tag_list)
        
        #create bee pairs list and colors if pairwise distance tracking is on
        if pairwise_distance_tracking == 'ON':
            bee_pairs = create_bee_pairs_list(tag_list)
            line_colors = create_line_colors(line_colors, bee_pairs)

    else:
        print("show_video is 'OFF\'")
   
    #extract filename and extension from video path
    filename, extension = extract_filename(video_path)

    #if write_video is on, extract size of video and create output file and video writer
    if write_video == 'ON':
        size = extract_size(vid)
        out = create_outputfile_and_videowriter(tracked_video_destination_directory, filename, video_filename_addition, extension, playback_framerate, size)


    #initialize variables in preparation for looping over frames in the video
    #one for storing the tracking trails, one for storing pairwise distances, and one for the frame number
    tracking_trail_dict = {}
    pairwise_trail_list = []
    frame_num = 0


    while(vid.isOpened()):
        ret, frame = vid.read() #ret stores boolean, frame stores the frame -code copied
        
        if ret == True:
            try:
                #store the original frame for later use when applying alpha blending
                frame_original = frame.copy()
                #This if statements lets you know you have to set one of these to ON         
                if write_video == 'ON' or show_video == 'ON':

                    #grabs the frame dataframe and the frame ids specifically 
                    frame_df = df[ df['frame'] == frame_num ]
                    frame_ids = frame_df['ID'].unique()

                    # draw connecting lines between bees in tag_list if pairwise_distance_tracking is ON
                    # if tracking_trail is on, have these lines persist for maybe five frames?
                    # I put this distance tracking code first so that the tag numbers and dots are drawn over the connecting lines
                    # this way they're seen more clearly
                    focal_bee_container = [] # will be updated to capture coordinates of focal bee and other tracked bee, in order to draw line between them
                    focal_bee_coord = None # will be updated to capture the coordinates of just the focal bee
                    contacts_container = [] # will be updated to capture the coordinates of bees that are in contact with each other

                    #To enter following code block, write or show video is set to on, and pairwise tracking is set to on
                    if pairwise_distance_tracking == 'ON':

                        for i in range(len(bee_pairs)):
                            bee1, bee2, bee1_loc, bee2_loc, distance = find_bee_pair_coordinates(i, bee_pairs, frame_ids, frame_df)

                            if bee1_loc != None and bee2_loc != None:
                                frame, focal_bee_coord, focal_bee_container, pairwise_trail_list, contacts_container = draw_bee_lines(i, pairwise_focal_bee, focal_bee_coord, bee1, bee2, bee1_loc, bee2_loc, pairwise_line_size, focal_bee_color, background_bee_color, line_colors, bee_pairs, frame, frame_num, pairwise_trail, pairwise_trail_list, focal_bee_container, distance, contacts, contact_distance, contacts_container, queen_id, queen_contact_distance)
                                
                        #Only plot the focal bee lines if there is a focal bee ID (its not set to 'OFF'), it's been tracked, and there are other tracked bees in the frame
                        if type(pairwise_focal_bee) == int and len(focal_bee_container) > 0: # only plot lines and calculate average distance vector if focal_bee is on and the focal bee and at least one other bee are tracked in the image
                            
                            frame = plot_focal_bee_lines(frame, focal_bee_container, focal_bee_coord, focal_bee_color, avg_distance_vector, avg_distance_vector_color)

                        #Only runs if you want a trail of pairwise tracks, and not just the lines between bees in the current frame - honestly doesnt look so great
                        if pairwise_trail == 'ON':            
                            frame = draw_pairwise_trail_lines(frame, pairwise_trail_list, frame_num)

                        #Add alpha blending to the frame if pairwise alpha is on, and save updated frame copy so we can add an alpha channel to the contacts lines
                        if enable_pairwise_alpha_channel == 'ON':
                            frame = cv2.addWeighted(frame, pairwise_alpha, frame_original, 1 - pairwise_alpha, 0)
                            updated_frame_copy = frame.copy()

                        #Draw the lines between contacting bees on top of all other lines
                        if contacts == 'ON':
                            frame = draw_contact_lines(contacts_container, contacts_color, contacts_line_size, enable_contacts_alpha_channel, contacts_alpha, frame, updated_frame_copy)

                    #Now we draw the tag ids, centroids, and tracking trails if its on
                    for id in frame_ids:
                        frame, tracking_trail_dict = draw_centroids_and_tracking_trails(frame, id, tag_list, frame_df, tracking_trail, tracking_trail_dict, tag_colors, frame_num, contacts, contacts_color, non_contact_color, contacts_container )
                        if contacts == "ON":
                            continue
                        frame = draw_ids(frame, id, frame_df, tag_list, tag_colors, contacts, contacts_container, contacts_color)
                else:
                    print('Both write_video and show_video are \'OFF\'')

            except Exception as e:
                print_error_message(e)





            #now just writing and showing the frame
            if write_video == 'ON':
                out.write(frame)

            if show_video == 'ON':
                cv2.imshow("Let's track these bees! Beep boop", frame)
    
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                cv2.waitKey(1)
            
            frame_num += 1

        #if ret is false, ie no more frames to read, break out of loop
        else:
            frame_num += 1
            break




    # Release everything once job is finished
    print("Done! Frames should be written to video.")
    vid.release()
    cv2.destroyAllWindows()

    return tag_colors, line_colors

In [24]:
#ATTENTION: This code works for MP4 and MJPEG video formats. If you have a different format, you may need to change the video writer codec and format.

csv_path = "/home/august/Desktop/Human Footprint/bumblebox-17_2025-02-14_17_30_03_tracking_interpolated_v2.csv"
video_path = '/home/august/Desktop/Human Footprint/bumblebox-17_2025-02-14_17_30_03.mp4'
tag_list = None #[22, 48, 43, 18, 32, 9, 25, 17, 40, 28, 11, 37, 24, 15, 42, 21, 30, 6, 31, 8, 2, 49, 0, 7, 35, 20, 19] # if None, it will track all tags
tag_colors = None #full_colony_colors #[(139,71,31,55),(),(),(),()] #full_colony_colors
show_video = 'OFF'
tracking_trail = 'OFF'
pairwise_distance_tracking = 'ON'
pairwise_trail = 'OFF'
line_colors = None

pairwise_focal_bee = 1 #'ON' # 'OFF' or tag number of focal bee, ex. 12
pairwise_line_size = 4
enable_pairwise_alpha_channel = 'ON'
pairwise_alpha = 0.3
focal_bee_color = (180,130,70)#(240,32,160) #(255, 100, 50, 150) #B,G,R #trying to add alpha #was (255, 50, 50)
background_bee_color = (180,130,70) # make this gray?

avg_distance_vector='OFF'
avg_distance_vector_color = (0,0,139) # possible values: 'complementary' (complementary to focal bee color), or some BGR color, ex (255, 255, 255), which is white 

contacts = 'ON' # With contacts on, lines will be drawn between bees that are within the contact distance
contact_distance = 175 # 150.4 #distance in pixels that is considered contact between two bees
queen_id = 0
queen_contact_distance = contact_distance * 1.35
contacts_color = (0, 0, 255) # BGR color for the contact lines
non_contact_color = (180,130,70)
contacts_line_size = 5
enable_contacts_alpha_channel = 'OFF' # If contacts are on, do you want to add an alpha channel to the contact lines to control how visible they are?
contacts_alpha = 0.9

write_video = 'ON' # Would you like to write the tracked frames that are displayed into a video? In order to write to a video, show_vid must be 'ON' - the only supported format at the moment is .mjpeg
video_filename_addition = '_distance_limit_v2' # If you're creating a video with tags tracked in it, what info do you want to add to your new filename to distinguish it from the video filename we're tracking? The default is to add '_tracked' to the end. For example, "col_20-2021-01-23_14-00-00.mjpeg" would become "col_20-2021-01-23_14-00-00_tracked.mjpeg"
#video_format = 'MP4' # tested with 'MJPEG' and 'MP4', the code will choose 'MJPEG' if nothing is specified - IF YOU SEE THIS ERROR: " OpenCV: FFMPEG: tag 0x67706a6d/'mjpg' is not supported with codec id 7 and format 'mjpeg / raw MJPEG video' " - check if the file was made, it worked for me on a Mac despite giving me this warning. If it doesn't work, try 'MP4'!
tracked_video_destination_directory = f"{os.path.dirname(video_path)}/" #Add a forward slash to the end of the video directory path
playback_framerate = 6

tag_id_colors = write_tags_onto_video(csv_path, video_path, tracked_video_destination_directory, playback_framerate=playback_framerate, 
                                                                                    show_video=show_video, 
                                                                                    write_video=write_video,  
                                                                                    tracking_trail=tracking_trail,
                                                                                    tag_list=tag_list,
                                                                                    tag_colors=tag_colors,
                                                                                    pairwise_distance_tracking=pairwise_distance_tracking,
                                                                                    line_colors=line_colors,
                                                                                    pairwise_trail=pairwise_trail,
                                                                                    pairwise_focal_bee=pairwise_focal_bee,
                                                                                    pairwise_line_size=pairwise_line_size,
                                                                                    enable_pairwise_alpha_channel=enable_pairwise_alpha_channel,
                                                                                    pairwise_alpha=pairwise_alpha,
                                                                                    focal_bee_color=focal_bee_color,
                                                                                    avg_distance_vector=avg_distance_vector,
                                                                                    avg_distance_vector_color=avg_distance_vector_color,
                                                                                    background_bee_color=background_bee_color,
                                                                                    contacts=contacts,
                                                         #initialize variables in preparation for looping over frames in the video
    #one for storing the tracking trails, one for storing pairwise distances, and one for the frame number
                                                                                    contact_distance=contact_distance,
                                                                                    queen_id=queen_id,
                                                                                    queen_contact_distance=queen_contact_distance,
                                                                                    contacts_color=contacts_color,
                                                                                    non_contact_color=non_contact_color,
                                                                                    contacts_line_size=contacts_line_size,
                                                                                    enable_contacts_alpha_channel=enable_contacts_alpha_channel,
                                                                                    contacts_alpha=contacts_alpha, 
                                                                                    video_filename_addition=video_filename_addition )

tag_list object created
[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(13), np.int64(14), np.int64(15), np.int64(17), np.int64(18), np.int64(20), np.int64(21), np.int64(22), np.int64(24), np.int64(25), np.int64(26), np.int64(31), np.int64(32), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(49)]
bumblebox-17_2025-02-14_17_30_03
write_video is on
/home/august/Desktop/Human Footprint/bumblebox-17_2025-02-14_17_30_03_distance_limit_v2.mp4
Contact between bees 0 and 38 at frame 0: distance is 228.60008748904713, contact distance is 236.25000000000003
Contact between bees 9 and 41 at frame 0: distance is 141.59802258506295, contact distance is 175
Contact between bees 13 and 46 at frame 0: distance is 156.01281998605114, contact d

In [ ]:
#This portion of the code focuses on drawing the centroids of potential tag IDs, for error checking purposes
'''
def draw_noID_centroids(frame, frame_df):
    
    for index, row in frame_df.iterrows():
    
        xmean = row['centroidX'] #int(ser.iloc[0])
        xmean = int(xmean)
        ymean = row['centroidY']
        ymean = int(ymean)
    
        
        cv2.circle(frame, tuple([int(xmean), int(ymean)]), 20, (0,0,255), -1)

        return frame

# Draw noIDs onto video to show the bees that were not detected in the frame
noID_path = '/home/august/Desktop/Human Footprint/bumblebox-11_2024-08-27_01_20_03_noID.csv'
video_path = '/home/august/Desktop/Human Footprint/bumblebox-11_2024-08-27_01_20_03_tracked.mp4'
noID_filename_addition = '_noID_tracked'
noID_destination_directory = '/home/august/Desktop/Human Footprint/'
write_noID_video = 'ON'
show_noID_video = 'OFF'
noID_playback_framerate = 6


   #read in csv data and video
df = pd.read_csv(noID_path)
vid = cv2.VideoCapture(video_path)
   
    #extract filename and extension from video path
filename, extension = extract_filename(video_path)

size = extract_size(vid)
out = create_outputfile_and_videowriter(noID_destination_directory, filename, noID_filename_addition, extension, noID_playback_framerate, size)

frame_num = 0

while(vid.isOpened()):
        ret, frame = vid.read() #ret stores boolean, frame stores the frame -code copied
        
        if ret == True:
            try:
                frame_df = df[ df['frame'] == frame_num ]

                for row in frame_df:
                     frame = draw_noID_centroids(frame, frame_df)
                
            except Exception as e:
                print_error_message(e)
            
            #now just writing and showing the frame
            if write_noID_video == 'ON':
                out.write(frame)

            if show_noID_video == 'ON':
                cv2.imshow("Let's track these bees! Beep boop", frame)
    
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                cv2.waitKey(1)
            
            frame_num += 1

        #if ret is false, ie no more frames to read, break out of loop
        else:
            frame_num += 1
            break

# Release everything once job is finished
print("Done! Frames should be written to video.")
vid.release()
cv2.destroyAllWindows()

'''                

bumblebox-11_2024-08-27_01_20_03_tracked
write_video is on
/home/august/Desktop/Human Footprint/bumblebox-11_2024-08-27_01_20_03_tracked_noID_tracked.mp4
Done! Frames should be written to video.


In [ ]:
'''
#ATTENTION! This portion of the code is to interpolate missing data in a csv file prior to creating
#tracked videos. You don't need to run this if you have already interpolated your data. In order to run this script, you need to have
#the data_cleaning.py file in the same directory as this script.

import data_cleaning
import pandas as pd
import numpy as np

csv_path = "/home/august/Desktop/Human Footprint/bumblebox-17_2025-02-14_17_30_03_tracking.csv"
interpolated_csv_path = '/home/august/Desktop/Human Footprint/bumblebox-17_2025-02-14_17_30_03_tracking_interpolated.csv'

max_seconds_gap = 2
actual_frames_per_second = 4.5

df = pd.read_csv(csv_path)
interpolated_df = data_cleaning.interpolate(df, max_seconds_gap, actual_frames_per_second)
interpolated_df = data_cleaning.remove_jumps(interpolated_df)
interpolated_csv_path = csv_path.replace('.csv', '_interpolated.csv')
interpolated_df.to_csv(interpolated_csv_path, index=False)

'''